In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt
from keras.utils import to_categorical


Using TensorFlow backend.


In [2]:
import glob
import pickle
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import Adam, Nadam
import keras
import random 
from sklearn import metrics  


In [3]:
!ls

cnn_proj_yvg_testing  courses  examples  final_cnn_final_pickles.ipynb


# the *hopefully* final pickles are pickled locally
## we only load them here
## refer to `repickle_all_final.py` on Github for the source code for final pickle
Note: final pickle pulls the 'spectrum' key of all the `.1sec_hdpickles` and aggregates them

In [3]:
def valLoad():
    DATA_PATH = "cnn_proj_yvg_testing"

    with open(os.path.join(DATA_PATH, "val_1_sec_hd_feature.pickle"), 'rb') as jar:
        x_val = pickle.load(jar)

    with open(os.path.join(DATA_PATH, "val_1_sec_hd_label.pickle"), 'rb') as jar:
        y_val = pickle.load(jar)

    print(y_val.shape)
    
    # convert to one hot
    # note: only execute once! per session!
    y_val = np.reshape(y_val, (y_val.shape[0], -1))
    y_val = to_categorical(y_val)
    print(y_val.shape)
    
    return x_val, y_val

In [5]:
!ls cnn_proj_yvg_testing

saved_models  val_1_sec_hd_feature.pickle
subPickles    val_1_sec_hd_label.pickle


# BEGIN Only run ONCE per session!
## reshapes for the CNN

In [9]:


# # convert to one hot
# # note: only execute once! per session!
# y_val = np.reshape(y_val, (y_val.shape[0], -1))
# y_val = to_categorical(y_val)
# print(y_val.shape)


In [10]:

# # convert to one hot for training
# # note: only execute once!

# y_train = np.reshape(y_train, (y_train.shape[0], -1))
# y_train = to_categorical(y_train)
# print(y_train.shape)

# END: only run once per session


In [4]:

# new 1 sec pickles
librispeech_spectrogram_path = "librispeech_1sec_hdpickle"
librit_spectrogram_path = "librit_1sec_hdpickle"


In [7]:
!ls

cnn_proj_yvg_testing  courses  examples  final_cnn_final_pickles.ipynb


In [5]:
def modelConstruct():
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(4,4), input_shape=(201, 66,1),
                     kernel_initializer=keras.initializers.glorot_normal(),activation = 'relu'))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(4,4), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(4,4), 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(4,4),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(4,4),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Nadam(lr = 0.001),
                  metrics=['accuracy'])
    return model

In [6]:

def imNorm(arr):
    for i in range(arr.shape[0]):

        arr[i] -= np.min(arr[i])
        arr[i] = np.minimum(arr[i], 255)
    return arr

# Hyper params tuning 

## Kernel size

In [8]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import Adam, Nadam, Adadelta
import keras
# Function to create model, required for KerasClassifier
def create_model(kernel_size):
    print("Construct ",kernel_size)
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), input_shape=(201, 66,1),
                     kernel_initializer=keras.initializers.glorot_normal(),activation = 'relu'))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(kernel_size,kernel_size), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Nadam(lr = 0.001),
                  metrics=['accuracy'])
    return model
file_path = "cnn_proj_yvg_testing/subPickles"
x_name = 'x_train_splitted0.pickle'
file_name = os.path.join(file_path, x_name) 
with open(file_name, 'rb') as jar:
    x_train = pickle.load(jar)
x_train = imNorm(x_train)

y_name = 'y_train_splitted0.pickle'
file_name = os.path.join(file_path, y_name)
with open(file_name, 'rb') as jar:
    y_train = pickle.load(jar)
y_train = to_categorical(y_train, num_classes = 2)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# define the grid search parameters
kernel_size = [3,4,5]
param_grid = dict(kernel_size=kernel_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Construct  3
Epoch 1/10
5034/5034 [==============================] - 5s 953us/step - loss: 1.3143 - acc: 0.5572
Epoch 2/10
5034/5034 [==============================] - 3s 562us/step - loss: 0.6435 - acc: 0.6317
Epoch 3/10
5034/5034 [==============================] - 3s 556us/step - loss: 0.5952 - acc: 0.6790
Epoch 4/10
5034/5034 [==============================] - 3s 558us/step - loss: 0.5691 - acc: 0.7086
Epoch 5/10
5034/5034 [==============================] - 3s 559us/step - loss: 0.5215 - acc: 0.7418
Epoch 6/10
5034/5034 [==============================] - 3s 545us/step - loss: 0.5068 - acc: 0.7527
Epoch 7/10
5034/5034 [==============================] - 3s 546us/step - loss: 0.4649 - acc: 0.7813
Epoch 8/10
5034/5034 [==============================] - 3s 534us/step - loss: 0.4394 - acc: 0.7966
Epoch 9/10
5034/5034 [==============================] - 3s 534us/step - loss: 0.4061 - acc: 0.8184
Epoch 10/10
5034/5034 [==============================] - 1s 222us/step
Construct  3
Epoch 1/10
5

## Learn rate

In [15]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import Adam, Nadam, Adadelta,RMSprop, Adagrad, Adamax
import keras
# Function to create model, required for KerasClassifier
def create_model(optimizer):
    print("Construct ",optimizer)
    kernel_size = 3
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), input_shape=(201, 66,1),
                     kernel_initializer=keras.initializers.glorot_normal(),activation = 'relu'))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(kernel_size,kernel_size), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer(lr = 0.001),
                  metrics=['accuracy'])
    return model
file_path = "cnn_proj_yvg_testing/subPickles"
x_name = 'x_train_splitted0.pickle'
file_name = os.path.join(file_path, x_name) 
with open(file_name, 'rb') as jar:
    x_train = pickle.load(jar)
x_train = imNorm(x_train)

y_name = 'y_train_splitted0.pickle'
file_name = os.path.join(file_path, y_name)
with open(file_name, 'rb') as jar:
    y_train = pickle.load(jar)
y_train = to_categorical(y_train, num_classes = 2)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# define the grid search parameters
#optimizer = ['Adadelta','SGD','RMSprop','Adagrad','Adam','Adamax','Nadam']
optimizer = [Adadelta,SGD,RMSprop,Adagrad,Adam,Adamax,Nadam]

param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Construct  <class 'keras.optimizers.Adadelta'>
Epoch 1/10
5034/5034 [==============================] - 4s 737us/step - loss: 6.9726 - acc: 0.4605
Epoch 2/10
5034/5034 [==============================] - 3s 540us/step - loss: 6.4307 - acc: 0.4648
Epoch 3/10
5034/5034 [==============================] - 3s 539us/step - loss: 5.7711 - acc: 0.4746
Epoch 4/10
5034/5034 [==============================] - 3s 544us/step - loss: 5.4598 - acc: 0.4708
Epoch 5/10
5034/5034 [==============================] - 3s 536us/step - loss: 5.1481 - acc: 0.4807
Epoch 6/10
5034/5034 [==============================] - 3s 534us/step - loss: 5.0205 - acc: 0.4760
Epoch 7/10
5034/5034 [==============================] - 3s 534us/step - loss: 4.9117 - acc: 0.4921
Epoch 8/10
5034/5034 [==============================] - 3s 532us/step - loss: 4.8257 - acc: 0.4879
Epoch 9/10
5034/5034 [==============================] - 3s 533us/step - loss: 4.6369 - acc: 0.4897
Epoch 10/10
5034/5034 [==============================] - 1s 22

5034/5034 [==============================] - 3s 542us/step - loss: 0.6615 - acc: 0.6140
Epoch 3/10
5034/5034 [==============================] - 3s 537us/step - loss: 0.6024 - acc: 0.6724
Epoch 4/10
5034/5034 [==============================] - 3s 533us/step - loss: 0.5530 - acc: 0.7133
Epoch 5/10
5034/5034 [==============================] - 3s 536us/step - loss: 0.5150 - acc: 0.7453
Epoch 6/10
5034/5034 [==============================] - 3s 539us/step - loss: 0.4856 - acc: 0.7674
Epoch 7/10
5034/5034 [==============================] - 3s 537us/step - loss: 0.4355 - acc: 0.7966
Epoch 8/10
5034/5034 [==============================] - 3s 542us/step - loss: 0.4011 - acc: 0.8206
Epoch 9/10
5034/5034 [==============================] - 3s 540us/step - loss: 0.3680 - acc: 0.8395
Epoch 10/10
5034/5034 [==============================] - 1s 238us/step
Construct  <class 'keras.optimizers.Adamax'>
Epoch 1/10
5034/5034 [==============================] - 4s 835us/step - loss: 3.4158 - acc: 0.5401
Epoc

## Learn rate

In [16]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import RMSprop
import keras
# Function to create model, required for KerasClassifier
def create_model(learn_rate):
    print("Construct ",learn_rate)
    kernel_size = 3
    optimizer = RMSprop(learn_rate)
    
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), input_shape=(201, 66,1),
                     kernel_initializer=keras.initializers.glorot_normal(),activation = 'relu'))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(kernel_size,kernel_size), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = 'relu'))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model
file_path = "cnn_proj_yvg_testing/subPickles"
x_name = 'x_train_splitted0.pickle'
file_name = os.path.join(file_path, x_name) 
with open(file_name, 'rb') as jar:
    x_train = pickle.load(jar)
x_train = imNorm(x_train)

y_name = 'y_train_splitted0.pickle'
file_name = os.path.join(file_path, y_name)
with open(file_name, 'rb') as jar:
    y_train = pickle.load(jar)
y_train = to_categorical(y_train, num_classes = 2)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# define the grid search parameters
learn_rate = [1.0e-5,1.0e-4,1.0e-3,1.0e-2,1.0e-1]
param_grid = dict(learn_rate=learn_rate)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Construct  1e-05
Epoch 1/10
5034/5034 [==============================] - 5s 912us/step - loss: 5.4805 - acc: 0.4774
Epoch 2/10
5034/5034 [==============================] - 3s 542us/step - loss: 4.4859 - acc: 0.4883
Epoch 3/10
5034/5034 [==============================] - 3s 535us/step - loss: 3.7565 - acc: 0.5129
Epoch 4/10
5034/5034 [==============================] - 3s 535us/step - loss: 3.2203 - acc: 0.5179
Epoch 5/10
5034/5034 [==============================] - 3s 532us/step - loss: 2.7477 - acc: 0.5294
Epoch 6/10
5034/5034 [==============================] - 3s 532us/step - loss: 2.5078 - acc: 0.5250
Epoch 7/10
5034/5034 [==============================] - 3s 567us/step - loss: 2.1528 - acc: 0.5425
Epoch 8/10
5034/5034 [==============================] - 3s 565us/step - loss: 1.9321 - acc: 0.5451
Epoch 9/10
5034/5034 [==============================] - 3s 572us/step - loss: 1.6910 - acc: 0.5463
Epoch 10/10
5034/5034 [==============================] - 1s 262us/step
Construct  1e-05
Epoc

Epoch 4/10
5034/5034 [==============================] - 3s 531us/step - loss: 0.5343 - acc: 0.7237
Epoch 5/10
5034/5034 [==============================] - 3s 531us/step - loss: 0.4863 - acc: 0.7656
Epoch 6/10
5034/5034 [==============================] - 3s 529us/step - loss: 0.4290 - acc: 0.8008
Epoch 7/10
5034/5034 [==============================] - 3s 528us/step - loss: 0.3840 - acc: 0.8284
Epoch 8/10
5034/5034 [==============================] - 3s 522us/step - loss: 0.3619 - acc: 0.8425
Epoch 9/10
5034/5034 [==============================] - 3s 532us/step - loss: 0.3381 - acc: 0.8504
Epoch 10/10
5034/5034 [==============================] - 1s 240us/step
Construct  0.001
Epoch 1/10
5034/5034 [==============================] - 5s 980us/step - loss: 1.1909 - acc: 0.5739
Epoch 2/10
5034/5034 [==============================] - 3s 519us/step - loss: 0.6388 - acc: 0.6448
Epoch 3/10
5034/5034 [==============================] - 3s 526us/step - loss: 0.5638 - acc: 0.7060
Epoch 4/10
5034/5034 

## Activation function

In [17]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import RMSprop
import keras
# Function to create model, required for KerasClassifier
def create_model(activation):
    print("Construct ",activation)
    kernel_size = 3
    optimizer = RMSprop(0.001)
    
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), input_shape=(201, 66,1),
                     kernel_initializer=keras.initializers.glorot_normal(),activation = activation))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(kernel_size,kernel_size), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = activation))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = activation))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=keras.initializers.glorot_normal(seed=None),activation = activation))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model
file_path = "cnn_proj_yvg_testing/subPickles"
x_name = 'x_train_splitted0.pickle'
file_name = os.path.join(file_path, x_name) 
with open(file_name, 'rb') as jar:
    x_train = pickle.load(jar)
x_train = imNorm(x_train)

y_name = 'y_train_splitted0.pickle'
file_name = os.path.join(file_path, y_name)
with open(file_name, 'rb') as jar:
    y_train = pickle.load(jar)
y_train = to_categorical(y_train, num_classes = 2)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Construct  softmax
Epoch 1/10
5034/5034 [==============================] - 9s 2ms/step - loss: 0.6902 - acc: 0.5397
Epoch 2/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6891 - acc: 0.5453
Epoch 3/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6881 - acc: 0.5473
Epoch 4/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6813 - acc: 0.5548
Epoch 5/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6689 - acc: 0.5681
Epoch 6/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6628 - acc: 0.5814
Epoch 7/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6539 - acc: 0.6055
Epoch 8/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6478 - acc: 0.6208
Epoch 9/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.6416 - acc: 0.6244
Epoch 10/10
5034/5034 [==============================] - 2s 351us/step
Construct  softmax
Epoch 1/10
5034/50

5034/5034 [==============================] - 3s 545us/step - loss: 0.1958 - acc: 0.9253
Epoch 8/10
5034/5034 [==============================] - 3s 540us/step - loss: 0.1605 - acc: 0.9378
Epoch 9/10
5034/5034 [==============================] - 3s 540us/step - loss: 0.1400 - acc: 0.9462
Epoch 10/10
5034/5034 [==============================] - 1s 262us/step
Construct  sigmoid
Epoch 1/10
5034/5034 [==============================] - 6s 1ms/step - loss: 0.7246 - acc: 0.5072
Epoch 2/10
5034/5034 [==============================] - 3s 558us/step - loss: 0.6917 - acc: 0.5425
Epoch 3/10
5034/5034 [==============================] - 3s 554us/step - loss: 0.6834 - acc: 0.5526
Epoch 4/10
5034/5034 [==============================] - 3s 543us/step - loss: 0.6580 - acc: 0.5973
Epoch 5/10
5034/5034 [==============================] - 3s 542us/step - loss: 0.6469 - acc: 0.6178
Epoch 6/10
5034/5034 [==============================] - 3s 550us/step - loss: 0.6339 - acc: 0.6353
Epoch 7/10
5034/5034 [==========

## Init mode

In [18]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import RMSprop
import keras
# Function to create model, required for KerasClassifier
def create_model(init_mode):
    print("Construct ",init_mode)
    kernel_size = 3
    optimizer = RMSprop(0.001)
    activation = "tanh"
    
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), input_shape=(201, 66,1),
                     kernel_initializer=init_mode,activation = activation))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(kernel_size,kernel_size), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), 
                                     kernel_initializer=init_mode,activation = activation))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=init_mode,activation = activation))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=init_mode,activation = activation))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model
file_path = "cnn_proj_yvg_testing/subPickles"
x_name = 'x_train_splitted0.pickle'
file_name = os.path.join(file_path, x_name) 
with open(file_name, 'rb') as jar:
    x_train = pickle.load(jar)
x_train = imNorm(x_train)

y_name = 'y_train_splitted0.pickle'
file_name = os.path.join(file_path, y_name)
with open(file_name, 'rb') as jar:
    y_train = pickle.load(jar)
y_train = to_categorical(y_train, num_classes = 2)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Construct  uniform
Epoch 1/10
5034/5034 [==============================] - 7s 1ms/step - loss: 0.6343 - acc: 0.6246
Epoch 2/10
5034/5034 [==============================] - 3s 569us/step - loss: 0.4706 - acc: 0.7757
Epoch 3/10
5034/5034 [==============================] - 3s 565us/step - loss: 0.3858 - acc: 0.8317
Epoch 4/10
5034/5034 [==============================] - 3s 566us/step - loss: 0.3199 - acc: 0.8689
Epoch 5/10
5034/5034 [==============================] - 3s 582us/step - loss: 0.2554 - acc: 0.8957
Epoch 6/10
5034/5034 [==============================] - 3s 606us/step - loss: 0.2227 - acc: 0.9122
Epoch 7/10
5034/5034 [==============================] - 3s 616us/step - loss: 0.1762 - acc: 0.9311
Epoch 8/10
5034/5034 [==============================] - 3s 604us/step - loss: 0.1574 - acc: 0.9420
Epoch 9/10
5034/5034 [==============================] - 3s 606us/step - loss: 0.1248 - acc: 0.9523
Epoch 10/10
5034/5034 [==============================] - 1s 298us/step
Construct  uniform
Ep

5034/5034 [==============================] - 3s 573us/step - loss: 0.2752 - acc: 0.8897
Epoch 7/10
5034/5034 [==============================] - 3s 572us/step - loss: 0.2293 - acc: 0.9066
Epoch 8/10
5034/5034 [==============================] - 3s 564us/step - loss: 0.1936 - acc: 0.9231
Epoch 9/10
5034/5034 [==============================] - 3s 573us/step - loss: 0.1640 - acc: 0.9388
Epoch 10/10
5034/5034 [==============================] - 1s 268us/step
Construct  glorot_uniform
Epoch 1/10
5034/5034 [==============================] - 8s 2ms/step - loss: 0.6634 - acc: 0.6091
Epoch 2/10
5034/5034 [==============================] - 3s 564us/step - loss: 0.5238 - acc: 0.7342
Epoch 3/10
5034/5034 [==============================] - 3s 559us/step - loss: 0.4000 - acc: 0.8230
Epoch 4/10
5034/5034 [==============================] - 3s 553us/step - loss: 0.3115 - acc: 0.8695
Epoch 5/10
5034/5034 [==============================] - 3s 568us/step - loss: 0.2633 - acc: 0.8888
Epoch 6/10
5034/5034 [===

5034/5034 [==============================] - 3s 567us/step - loss: 0.1467 - acc: 0.9420
Epoch 10/10
5034/5034 [==============================] - 1s 273us/step
Construct  he_uniform
Epoch 1/10
5034/5034 [==============================] - 8s 2ms/step - loss: 0.7180 - acc: 0.5884
Epoch 2/10
5034/5034 [==============================] - 3s 549us/step - loss: 0.5670 - acc: 0.7014
Epoch 3/10
5034/5034 [==============================] - 3s 552us/step - loss: 0.4655 - acc: 0.7777
Epoch 4/10
5034/5034 [==============================] - 3s 584us/step - loss: 0.3936 - acc: 0.8270
Epoch 5/10
5034/5034 [==============================] - 3s 562us/step - loss: 0.3420 - acc: 0.8498
Epoch 6/10
5034/5034 [==============================] - 3s 562us/step - loss: 0.2817 - acc: 0.8868
Epoch 7/10
5034/5034 [==============================] - 3s 563us/step - loss: 0.2438 - acc: 0.9078
Epoch 8/10
5034/5034 [==============================] - 3s 574us/step - loss: 0.2049 - acc: 0.9184
Epoch 9/10
5034/5034 [=======

## Dropout rate

In [22]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import LeakyReLU
from keras.optimizers import RMSprop
import keras
# Function to create model, required for KerasClassifier
def create_model(dropout_rate):
    print("Construct ",dropout_rate)
    kernel_size = 3
    optimizer = RMSprop(0.001)
    activation = "tanh"
    init_mode = "glorot_normal"
    
    model = Sequential()
    
    ###### conv layer 1 ######
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), input_shape=(201, 66,1),
                     kernel_initializer=init_mode,activation = activation))

    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(kernel_size,kernel_size), padding='SAME'))

    
    ###### conv 2 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size), 
                                     kernel_initializer=init_mode,activation = activation))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))

    
    ###### conv 3 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=init_mode,activation = activation))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ###### conv 4 ##########
    model.add(Conv2D(32, kernel_size=(kernel_size,kernel_size),strides=(1, 1), padding='SAME', 
                                     kernel_initializer=init_mode,activation = activation))

    #model.add(LeakyReLU(alpha=0.1))
    keras.layers.BatchNormalization(axis = 0)
    model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
    
    ### Fully connected #######
    model.add(Flatten())
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(64, activation=''relu'))

    model.add(keras.layers.Dropout(dropout_rate, noise_shape=None, seed=None))
    model.add(Dense(2, activation='softmax'))
    
    # compile and return 
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model
file_path = "cnn_proj_yvg_testing/subPickles"
x_name = 'x_train_splitted0.pickle'
file_name = os.path.join(file_path, x_name) 
with open(file_name, 'rb') as jar:
    x_train = pickle.load(jar)
x_train = imNorm(x_train)

y_name = 'y_train_splitted0.pickle'
file_name = os.path.join(file_path, y_name)
with open(file_name, 'rb') as jar:
    y_train = pickle.load(jar)
y_train = to_categorical(y_train, num_classes = 2)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=1)
# define the grid search parameters
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Construct  0.0
Epoch 1/10
5034/5034 [==============================] - 8s 2ms/step - loss: 0.6281 - acc: 0.6379
Epoch 2/10
5034/5034 [==============================] - 3s 559us/step - loss: 0.4551 - acc: 0.7888
Epoch 3/10
5034/5034 [==============================] - 3s 560us/step - loss: 0.3411 - acc: 0.8526
Epoch 4/10
5034/5034 [==============================] - 3s 563us/step - loss: 0.2544 - acc: 0.9009
Epoch 5/10
5034/5034 [==============================] - 3s 556us/step - loss: 0.2011 - acc: 0.9203
Epoch 6/10
5034/5034 [==============================] - 3s 560us/step - loss: 0.1608 - acc: 0.9368
Epoch 7/10
5034/5034 [==============================] - 3s 562us/step - loss: 0.1225 - acc: 0.9563
Epoch 8/10
5034/5034 [==============================] - 3s 604us/step - loss: 0.0963 - acc: 0.9670
Epoch 9/10
5034/5034 [==============================] - 3s 618us/step - loss: 0.0752 - acc: 0.9756
Epoch 10/10
5034/5034 [==============================] - 1s 289us/step
Construct  0.0
Epoch 1/10

5034/5034 [==============================] - 3s 577us/step - loss: 0.2753 - acc: 0.8858
Epoch 8/10
5034/5034 [==============================] - 3s 581us/step - loss: 0.2430 - acc: 0.8997
Epoch 9/10
5034/5034 [==============================] - 3s 577us/step - loss: 0.2107 - acc: 0.9138
Epoch 10/10
5034/5034 [==============================] - 1s 285us/step
Construct  0.5
Epoch 1/10
5034/5034 [==============================] - 9s 2ms/step - loss: 0.6788 - acc: 0.5981
Epoch 2/10
5034/5034 [==============================] - 3s 582us/step - loss: 0.5358 - acc: 0.7306
Epoch 3/10
5034/5034 [==============================] - 3s 570us/step - loss: 0.4614 - acc: 0.7859
Epoch 4/10
5034/5034 [==============================] - 3s 566us/step - loss: 0.3889 - acc: 0.8266
Epoch 5/10
5034/5034 [==============================] - 3s 576us/step - loss: 0.3457 - acc: 0.8449
Epoch 6/10
5034/5034 [==============================] - 3s 586us/step - loss: 0.3121 - acc: 0.8667
Epoch 7/10
5034/5034 [==============

5034/5034 [==============================] - 3s 595us/step - loss: 0.7214 - acc: 0.5677
Epoch 3/10
5034/5034 [==============================] - 3s 587us/step - loss: 0.6672 - acc: 0.6271
Epoch 4/10
5034/5034 [==============================] - 3s 590us/step - loss: 0.6335 - acc: 0.6482
Epoch 5/10
5034/5034 [==============================] - 3s 607us/step - loss: 0.6016 - acc: 0.6768
Epoch 6/10
5034/5034 [==============================] - 3s 606us/step - loss: 0.5741 - acc: 0.7074
Epoch 7/10
5034/5034 [==============================] - 3s 612us/step - loss: 0.5261 - acc: 0.7493
Epoch 8/10
5034/5034 [==============================] - 3s 589us/step - loss: 0.4905 - acc: 0.7714
Epoch 9/10
5034/5034 [==============================] - 3s 599us/step - loss: 0.4683 - acc: 0.7825
Epoch 10/10
5034/5034 [==============================] - 1s 287us/step
Construct  0.2
Epoch 1/10
7551/7551 [==============================] - 12s 2ms/step - loss: 0.6192 - acc: 0.6561
Epoch 2/10
7551/7551 [=============

# End of hyper params tuning